In [112]:
import telebot
import pandas as pd
import numpy as np
import random
from telebot import types
import sqlite3
import re
import os
from pydub import AudioSegment


#0. добавить отправление сообщения пользователю
#1.добавить в базу данных ссылку на файл от пользователя - колонка 
#+3.научиться загружать аудиозаписи от пользователей., спрашивать пользователя если в базе несколько поздравлений с ним 
#+кого он пришел поздравлять (показывать ему ники)
#4.найти библиотеку чтобы склеивать записи, склеить записи, возможно добавить еще колонку что запись сделана
#5.отослать поздравление использовать библиотеку time

#логирование
#import logging
#logger=telebot.logger
#telebot.logger.setLevel(logging.DEBUG)

bot= telebot.TeleBot('1236450831:AAFvNbHUO5fkEsb1iCh3DA9xL_1oqkyq9pM')

status=''
owner_id=''
congratulation=0
name_for_congr='___'
name_for_voice=''
owner_data=''
file_id_info=''
who_send=''
id_congr_for_audio=''

@bot.message_handler(func=lambda message: message.text== 'Привет' or message.text== 'привет'  )
def get_callback_query(message): 
    if message.text == "Привет" or message.text == "привет":
        bot.send_message(message.from_user.id, 'Привет я бот поздравлений, я помогу сделать тебе коллективное аудиопоздравление') 
        global owner_data
        owner_data=message.from_user.username +' '+ str(message.from_user.id)
        in_keyboard=generate_inline_keyboard([['Примеры поздравлений','Примеры поздравлений'],['Как это работает','Как это работает'],['Приступить к поздравлению',owner_data]])
        
        bot.send_message(message.from_user.id,'Используй кнопки', reply_markup=in_keyboard)
    else:
        bot.send_message(message.from_user.id, "Я тебя не понимаю. Напиши - привет")
        
@bot.message_handler(content_types=['text']) #ввод
def get_text_message(message):
    global owner_id
    global status
    global congratulation

    name=str(re.findall('@'+r'\w',message.text)) #используем регулярное выражение. оно берет @ и первую букву ника
    name=name[2:4]
    #print('обработанное имя',name)
    #print('первые две буквы имени',message.text[0:2])
    if name == message.text[0:2]: 
        owner_id=message.from_user.id
        status='рядовой'
        insert_to_database(message.text,status,message.from_user.id,congratulation)
    elif '@@' in message.text:
        owner_id=message.from_user.id
        status='одоряемый'
        insert_to_database(message.text,status,message.from_user.id,congratulation)
         
    else:
        bot.send_message(message.from_user.id,'Ты хочешь собрать друзей и поздравить человека напиши - привет')
        bot.send_message(message.from_user.id,'Если ты просто хочешь записать аудио, просто нажми на микрофон и запиши аудио как обычно, а все остальное я сделаю сам')

        
@bot.callback_query_handler(func=lambda call:True) # ответ на ввд
def call_back(call):
    global name_for_congr
    global name_for_voice
    global owner_data
    global id_congr_for_audio

    if call.data=='Как это работает':
        bot.send_message(call.message.chat.id,'1. Отметить всех участников поздравления\n2. Бот отправит участникам приглашение для поздравления\n3. Участники поздравления запишут 10 сикундную аудиозапись\n4. Бот скомпанует аудиозапись и наложит музыку\n5. Бот отправит адресату в нужный день и позвонит при необходимост\n6. Участники поздравления получат полную запись и обратную связь от одоряемого')
    elif call.data=='Примеры поздравлений':
        bot.send_message(call.message.chat.id, 'Прослушай пример поздравления')        
        audio=open('/home/mikhail/09.06.2020.mp3','rb')
        bot.send_audio(call.message.chat.id,audio)

        
    elif call.data==owner_data:
        congratulation=congratulation_count()
        bot.send_message(call.message.chat.id, 'отошли людям которые будут участвовать в поздравлении название бота, только так они смогут сюда попасть:')
        bot.send_message(call.message.chat.id, '@toll_toll_happy_bot')

        bot.send_message(call.message.chat.id, 'Напиши ники людей с которыми ты хочешь участвовать в поздравлении, обязательно начинай ники с символа @, пожалуйста каждого нового человека пришли новым сообщением, образец - @Никнейм')
        bot.send_message(call.message.chat.id, 'Напиши ник человека, которого ты хочешь поздравить, вот образец -  @@Никнейм')
        status='начальник'
        for i in range(len(call.data)):
            if call.data[i]==' ':
                name='@'+call.data[0:i]
                id_=call.data[i+1::]
        insert_to_database(name,status,id_,congratulation)
    
    else:
        # определим имя директории, которую создаём 
        global who_send
        global file_id_info 
        bot.send_message(call.message.chat.id, f'я добавил твою запись в папку {call.data}')

        path = f"Все поздравления/{call.data}"
        try:
            os.makedirs(path)
        except OSError:
            print ("Создать директорию %s не удалось" % path)
        else:
            print ("Успешно создана директория %s" % path)     
        download_file=bot.download_file(file_id_info.file_path)
        with open(os.path.join(f'Все поздравления/{call.data}', f'поздравление от {who_send}.wav'), 'wb') as f: 
            f.write(download_file)
            
     
        insert_audio_path_to_database(f'Все поздравления/{call.data}',id_congr_for_audio,'@'+f'{who_send}') 
        
        conn=sqlite3.connect('Happy_Users.db')
        c=conn.cursor()
        conn.close    
        data=pd.read_sql('SELECT * from Happy_Users',conn)
        ss=0
        for i in range(len(data.index)):
            buffer= data[data['Congratulations_ID'==i]
            buffer.index=np.arange(len(buffer))
            for j in range(len(buffer.index)):   
                if 'Все' in buffer.loc[j,'Path_to_Audio']:
                    ss+=1
                    if len(buffer.index) == (ss-1):
                         #Склеиваем запись
                        sound1 = AudioSegment.from_wav("filename01.wav")
                        sound2 = AudioSegment.from_wav("filename02.wav")
                        combined_sounds = sound1 + sound2  
                        combined_sounds.export("joinedFile.wav", format="wav")
                         
                         

                   
            

    

        

    
#{'file_id': 'AwACAgIAAxkBAAII8F7onjNKM78N5P_jrEnPsTXQRtJeAAKuBQACfsdIS1VR_2MMuskpGgQ',
#'duration': 0, 'performer': None, 'title': None, 'mime_type': 'audio/ogg', 'file_size': 3022}
@bot.message_handler(content_types=['voice'])
def get_audio_message_from_owner(message):
    global who_send
    global file_id_info
    global id_congr_for_audio

    congratulation=congratulation_count()
    conn=sqlite3.connect('Happy_Users.db')
    c=conn.cursor()
    conn.close    
    data=pd.read_sql('SELECT * from Happy_Users',conn)
    print('Упоминания пользователя в бд',data[data['User_Name'] == '@'+message.from_user.username])
    
    #ищем людей для которых мы указаны как поздравитель
    who_send=message.from_user.username
    user=data[data['User_Name'] == '@' + message.from_user.username]
    user.index = np.arange(len(user))
    ii=0
    print(user)
    file_id_info=bot.get_file(message.voice.file_id)
    
    for i in range(len(data.index)):
        for j in range(len(user.index)):
            if data.loc[i,'Congratulations_ID']==user.loc[j,'Congratulations_ID']:
                if data.loc[i,'Status'] == 'одоряемый':
                    id_congr_for_audio=user.loc[j,'Congratulations_ID']
                    name_for_congr=data.loc[i,'User_Name']
                    name_for_congr=name_for_congr[2::]
                    in_keyboard1= types.InlineKeyboardMarkup()
                    button = types.InlineKeyboardButton(text=f'Выбрать {name_for_congr} для поздралвения',callback_data=name_for_congr)
                    in_keyboard1.add(button)
                    bot.send_message(message.from_user.id,'Кому адресовано это аудиосообщение', reply_markup=in_keyboard1)
    



        
#делаем инлайн клавиатуру
def generate_inline_keyboard(list_=[]):
    in_keyboard= types.InlineKeyboardMarkup()
    for i in list_:
        button = types.InlineKeyboardButton(text=i[0],callback_data=i[1])
        in_keyboard.add(button)
    return in_keyboard
    
#делаем обычную кнопку клавиатуру
def generate_keyboard(*args):
    keyboard= types.ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    for i in args:
        button = types.KeyboardButton(i)
        keyboard.add(button)
    return keyboard
#keyboard=generate_keyboard('записать аудио', 'поздравить')
#bot.send_message(message.chat.id,msg,reply_markup=keyboard)

#добавляем в базу поздравителей
def insert_to_database(msg,status,owner_id,congratulation,path_to_audio='_'):
    conn=sqlite3.connect('Happy_Users.db')
    c=conn.cursor()
    param=[(msg,status,owner_id,congratulation,path_to_audio)]
    c.executemany("INSERT INTO Happy_Users VALUES (?,?,?,?,?)", param)
    conn.commit()
    conn.close
    
    data=pd.read_sql('SELECT * from Happy_Users',conn)
    print(data)
    
def insert_audio_path_to_database(path_to_audio, congr_id_for_audio, user_name_for_audio):
    path_to_audio="'"+str(path_to_audio)+"'"
    
    user_name_for_audio="'"+(user_name_for_audio)+"'"
    

    conn=sqlite3.connect('Happy_Users.db')
    c=conn.cursor()
    sql=f""" UPDATE Happy_Users SET Path_to_Audio = {path_to_audio} WHERE User_Name= {user_name_for_audio} AND Congratulations_ID={congr_id_for_audio}"""
    c.execute(sql)
    conn.commit()
    conn.close
    
    data=pd.read_sql('SELECT * from Happy_Users',conn)
    print(data)
    

    
#подсчитываем количество начальников поздравлений и по сути присваиваем каждой группе поздравляющих свой номер
def congratulation_count():
    global congratulation
    conn=sqlite3.connect('Happy_Users.db')
    c=conn.cursor()
    conn.close    
    data=pd.read_sql('SELECT * from Happy_Users',conn)
    congratulation=len(data[data['Status']=='начальник'].index)
    return congratulation


bot.polling(none_stop=True, interval=0)

2020-06-17 13:36:46,467 (util.py:68 PollingThread) ERROR - TeleBot: "ApiException occurred, args=('A request to the Telegram API was unsuccessful. The server returned HTTP 409 Conflict. Response body:\n[b\'{"ok":false,"error_code":409,"description":"Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"}\']',)
Traceback (most recent call last):
  File "/home/mikhail/bot_dev/lib64/python3.8/site-packages/telebot/util.py", line 62, in run
    task(*args, **kwargs)
  File "/home/mikhail/bot_dev/lib64/python3.8/site-packages/telebot/__init__.py", line 281, in __retrieve_updates
    updates = self.get_updates(offset=(self.last_update_id + 1), timeout=timeout)
  File "/home/mikhail/bot_dev/lib64/python3.8/site-packages/telebot/__init__.py", line 251, in get_updates
    json_updates = apihelper.get_updates(self.token, offset, limit, timeout, allowed_updates)
  File "/home/mikhail/bot_dev/lib64/python3.8/site-packages/telebot/apihelper.py", line 199,

    User_Name     Status   Owner_ID  Congratulations_ID Path_to_Audio
0  @Lostbrown  начальник  440324273                   0             _
1  @Lostbrown  начальник  440324273                   1             _
    User_Name     Status   Owner_ID  Congratulations_ID Path_to_Audio
0  @Lostbrown  начальник  440324273                   0             _
1  @Lostbrown  начальник  440324273                   1             _
2        @лол    рядовой  440324273                   1             _
    User_Name     Status   Owner_ID  Congratulations_ID Path_to_Audio
0  @Lostbrown  начальник  440324273                   0             _
1  @Lostbrown  начальник  440324273                   1             _
2        @лол    рядовой  440324273                   1             _
3      @хвост    рядовой  440324273                   1             _
    User_Name     Status   Owner_ID  Congratulations_ID Path_to_Audio
0  @Lostbrown  начальник  440324273                   0             _
1  @Lostbrown  начал

In [110]:
import sqlite3

conn=sqlite3.connect('Happy_Users.db')

c=conn.cursor()
c.execute('''CREATE TABLE Happy_Users ('User_Name','Status','Owner_ID','Congratulations_ID','Path_to_Audio')''')
conn.commit()
conn.close
data=pd.read_sql('SELECT * from Happy_Users',conn)
print(data)


Empty DataFrame
Columns: [User_Name, Status, Owner_ID, Congratulations_ID, Path_to_Audio]
Index: []


In [106]:
name=str(re.findall('@'+r'\w','@Lostbrown')) #используем регулярное выражение . оно берет @ и первую букву ника
name=name[2:4]
name

'@L'

In [87]:
call='Lostbrown'+' '+'440324273'
for i in range(len(call)):
    if call[i]==' ':
        name='@'+call[0:i]
        id_=call[i+1::]

'440324273'

In [101]:
a='@@L'
a[0:2]

'@@'

In [125]:
data=pd.read_sql('SELECT * from Happy_Users',conn)
len(data[data['Status']=='начальник поздравления'].index)


5

NameError: name 'user' is not defined

In [31]:
def xx(x):
    x=1
    
x=xx(1)
print(x)

None


In [39]:
import os
os.mkdir('kk')